# Kafka Connect

Kafka Connect - это стандартный способ расширения возможностей Kafka для работы с источниками и приемниками данных.

Всех клиентов Apache Kafka можно отнести либо к категории консьюмеров, либо к категории продюсеров, и по большей части вся работа с Apache Kafka сводится к загрузке данных в топики или выгрузке данных из топиков во внешние системы: базы данных, файловые системы, объектные хранилища и т.д. Kafka Connect инкапсулирует лучше практики работы со стандартными источниками и приемниками с целью избавить программистов от процесса разработки и отладки своих продюсеров и консьюмеров: можно взять готовое решение и использовать его, сокращая таким образом TTM (time-to-market).

Компания Confluent поддерживает репозиторий [Confluent Hub](https://www.confluent.io/hub/) с богатым выбором Production Ready коннекторов, которые можно свободно использовать в промышленном окружении.

Любой проект может развенуть свой сервис Kafka Connect, который представляет собой отдельно запущенный сервер, на котором находятся jar файлы с разными типами коннекторов, доступных для внедрения. Также Kafka Connect предлагает удобный REST API интерфейс для управления запущенными коннекторами.

## Получение списка доступных коннекторов

Проверить список доступных плагинов можно при помощи REST API:

In [ ]:
curl -s http://connect:8083/connector-plugins | json_pp

## Установка новых коннекторов

Коннекторы представляют собой jar файлы, которые можно запустить со специальной конфигурацией. Коннекторы можно положить на сервер Kafka Connect через:
- копирование jar файла на сервер,
- утилиту [confluent-hub](https://docs.confluent.io/platform/current/connect/confluent-hub/command-reference/index.html).

Утилита [confluent-hub](https://docs.confluent.io/platform/current/connect/confluent-hub/command-reference/index.html) позволяет скачать коннектор с Confluent Hub и автоматически положить его файлы в корректные директории

Для примера будет использоваться [JDBC Connector](https://www.confluent.io/hub/confluentinc/kafka-connect-jdbc):

In [ ]:
confluent-hub install --no-prompt confluentinc/kafka-connect-jdbc:10.7.3

После установки плагина необходимо перезапустить Kafka Connect сервер:

In [ ]:
docker compose restart connect

Сервису Kafka Connect необходимо время чтобы запуститься:

In [ ]:
docker compose ps connect | grep -q healthy || {
    echo 'Kafka Connect еще не запустился. Пожалуйста, подождите' >&2
    false
} && {
    echo 'Kafka Connect доступен для работы!'
}

Убедиться, что плагин установлен:

In [ ]:
curl -s http://connect:8083/connector-plugins | json_pp

Можно заметить, что установилось два коннектора:

- sink - приемник данных, т.е. выгружает данные из топика во внешнюю систему (база данных в данном случае);
- source - источник данных, т.е. загружает данные из внешней системы в топик в Apache Kafka.

## Подготовка

Коннектор будет забирать данные из таблицы `products`:

In [ ]:
psql -U postgres -d postgres <<EOF
CREATE TABLE IF NOT EXISTS products(
    id INT PRIMARY KEY,
    name VARCHAR(255),
    price INT
);

\d products
EOF

In [ ]:
psql -U postgres -d postgres <<EOF
insert into products values(1, 'Cheese', 5);
insert into products values(2, 'Milk', 3);
insert into products values(3, 'Meat', 7);
insert into products values(4, 'Eggs', 4);
EOF

In [ ]:
psql -U postgres -d postgres <<EOF
select * from products
EOF

## Создание коннектора-продюсера (source)

### Конфигурация коннектора

In [ ]:
cat > /tmp/postgres-source-connector-1.json <<EOF
{
  "name": "jdbc-source-connector-1", 
  "config": 
    {"connector.class": "JdbcSource", 
     "tasks.max": 1,
     "topic.prefix": "connect-jdbc-1-",
     "connection.url": "jdbc:postgresql://postgres:5432/postgres",
     "mode": "incrementing", 
     "incrementing.column.name":"id",
     "value.converter": "org.apache.kafka.connect.json.JsonConverter",
     "value.converter.schemas.enable":"true", 
     "table.whitelist" : "public.products",
     "connection.user": "postgres", 
     "connection.password" : "postgres"
    }
}    
EOF

### Запуск коннектора

In [ ]:
curl -s http://connect:8083/connectors \
    -X POST \
    -H 'Content-Type: application/json' \
    -d '@/tmp/postgres-source-connector-1.json' | json_pp

### Проверка статуса коннектора

In [ ]:
curl -s http://connect:8083/connectors/jdbc-source-connector-1/status | json_pp

### Проверка работы коннектора

In [ ]:
kafka-console-consumer \
    --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic connect-jdbc-1-products \
    --from-beginning \
    --timeout-ms 10000 |
    while read line ; do json_pp <<<"$line" ; done

## Cоздание коннектора-продюсера с преобразованием полей (SMT)

Иногда бывает необхоимо преобразовать данные перед загрузкой или выгрузкой из Apache Kafka. Для этих целей подходит [SMT - Single Message Transformer](https://docs.confluent.io/platform/current/connect/transforms/overview.html). При помощи SMT можно настроить:
- преобразование данных,
- удаление колонок,
- приведение типов,
- и т.д.

In [ ]:
cat > /tmp/postgres-source-connector-2.json <<EOF
{
  "name": "jdbc-source-connector-2",
  "config":
    {"connector.class": "JdbcSource",
     "tasks.max": 1,
     "topic.prefix": "connect-jdbc-2-",
     "connection.url": "jdbc:postgresql://postgres:5432/postgres",
     "mode": "incrementing",
     "incrementing.column.name":"id",
     "value.converter": "org.apache.kafka.connect.json.JsonConverter",
     "value.converter.schemas.enable":"true",
     "table.whitelist" : "public.products",
     "connection.user": "postgres",
     "connection.password" : "postgres",
     "transforms" : "createKey,extractInt,RenameField",
     "transforms.createKey.type":"org.apache.kafka.connect.transforms.ValueToKey",
     "transforms.createKey.fields":"id",
     "transforms.extractInt.type":"org.apache.kafka.connect.transforms.ExtractField\$Key",
     "transforms.extractInt.field":"id",
     "transforms.RenameField.type" : "org.apache.kafka.connect.transforms.ReplaceField\$Value",
     "transforms.RenameField.renames" : "name:code"
    }
}
EOF

In [ ]:
curl -s http://connect:8083/connectors \
    -X POST \
    -H 'Content-Type: application/json' \
    -d '@/tmp/postgres-source-connector-2.json' | json_pp

In [ ]:
kafka-console-consumer \
    --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic connect-jdbc-2-products \
    --property print.key=true \
    --from-beginning \
    --timeout-ms 10000

Можно заметить, что у сообщений появился ключ

## Создание коннектора-консьюмера (sink)

Коннекторы могут так же выгружать данные из Kafka во внешние системы, например, в базу данных:

In [ ]:
cat > /tmp/postgres-sink-connector-1.json <<EOF
{
  "name": "jdbc-sink-connector",
  "config": {
    "connector.class": "io.confluent.connect.jdbc.JdbcSinkConnector",
    "tasks.max": "1",
    "topics": "customers",
    "connection.url": "jdbc:postgresql://postgres:5432/postgres",
    "connection.user": "postgres",
    "connection.password": "postgres",
    "connection.ds.pool.size": 5,
    "auto.create": "true",
    "insert.mode.databaselevel": true,
    "value.converter": "io.confluent.connect.avro.AvroConverter",
    "value.converter.schema.registry.url": "http://schema-registry:8081"
  }
}
EOF

In [ ]:
curl -s http://connect:8083/connectors \
    -X POST \
    -H 'Content-Type: application/json' \
    -d '@/tmp/postgres-sink-connector-1.json' | json_pp

Для работы коннектора необходимо отправить сообщения в топик в формате `AVRO`, это можно сделать при помощи `kafka-avro-console-producer`:

Подготовка схемы:

In [ ]:
cat <<EOF > /tmp/customer.avsc
{
   "fields" : [
      {
         "name" : "id",
         "type" : "int"
      },
      {
         "name" : "name",
         "type" : "string"
      }
   ],
   "name" : "customer",
   "type" : "record"
}
EOF

Отравка данных:

In [ ]:
kafka-avro-console-producer \
    --bootstrap-server kafka:9092 \
    --topic customers  \
    --property schema.registry.url=http://schema-registry:8081 \
    --property value.schema="$(cat /tmp/customer.avsc | single_line | escape_double_qoutes | skip_ws)" \
<<EOF
{"id": $RANDOM, "name": "Jane Doe"}
{"id": $RANDOM, "name": "John Smith"}
{"id": $RANDOM, "name": "Ann Black"}
EOF

Проверка, что сообщения доставлены:

In [ ]:
kafka-avro-console-consumer \
    --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --enable-systest-events false \
    --topic customers \
    --timeout-ms 10000 \
    --from-beginning

Проверка работы коннектора: данные должны попасть в базу данных в таблицу `customers`:

In [ ]:
psql -U postgres -d postgres <<EOF
SELECT *
  FROM customers
EOF

Таблица `consumers` была создана автоматически, но у нее абсолютно нет никакой мета-информации, например, отсутствует первичный ключ:

In [ ]:
psql -U postgres -d postgres <<EOF
\d customers
EOF

## Создание коннектор-консьюмера (sink) с трансформациями

При чтении данных можно добавлять метаданные, например, указать какое поле будет первичным ключом:

In [ ]:
cat > /tmp/postgres-sink-connector-2.json <<EOF
{
  "name": "jdbc-sink-connector-2",
  "config": {
    "connector.class": "io.confluent.connect.jdbc.JdbcSinkConnector",
    "tasks.max": "1",
    "topics": "customers",
    "connection.url": "jdbc:postgresql://postgres:5432/postgres",
    "connection.user": "postgres",
    "connection.password": "postgres",
    "connection.ds.pool.size": 5,
    "auto.create": "true",
    "table.name.format": "kafka_\${topic}",
    "insert.mode.databaselevel": true,
    "value.converter": "io.confluent.connect.avro.AvroConverter",
    "value.converter.schema.registry.url": "http://schema-registry:8081",
    "pk.mode" : "record_value",
    "pk.fields": "id"
  }
}
EOF

In [ ]:
curl -s http://connect:8083/connectors \
    -X POST \
    -H 'Content-Type: application/json' \
    -d '@/tmp/postgres-sink-connector-2.json' | json_pp

In [ ]:
kafka-avro-console-producer \
    --bootstrap-server kafka:9092 \
    --topic customers  \
    --property schema.registry.url=http://schema-registry:8081 \
    --property value.schema="$(cat /tmp/customer.avsc | single_line | escape_double_qoutes | skip_ws)" <<EOF
{"id": $RANDOM, "name": "Василий Тёркин"}
{"id": $RANDOM, "name": "Барон Мюнхаузен"}
EOF

In [ ]:
psql -U postgres -d postgres <<EOF
SELECT *
  FROM kafka_customers
EOF

In [ ]:
psql -U postgres -d postgres <<EOF
\d kafka_customers
EOF

Можно заметить, что создалась таблица `kafka_customers` с первичным ключом.

## Управление состоянием коннектора

### Приостановка коннкетора

Коннектор можно поставить на паузу:

In [ ]:
curl -v -X PUT http://connect:8083/connectors/jdbc-source-connector-1/pause

Убедиться, что коннектор приостановлен:

In [ ]:
curl -s -X GET http://connect:8083/connectors/jdbc-source-connector-1/status | json_pp

### Возобновление работы коннектора

In [ ]:
curl -v -X PUT http://connect:8083/connectors/jdbc-source-connector-1/resume

In [ ]:
curl -s -X GET http://connect:8083/connectors/jdbc-source-connector-1/status | json_pp

### Выводы

1. Kafka Connect упрощает реализацию стандартных сценариев при работе с Apache Kafka:
    - загрузить в топик данные из внешней системы,
    - выгрузить данные из топика во внешнюю систему.
1. Kafka Connect предлагает REST API для работы с коннекторами;
1. Коннекторы являются jar файлами, которые можно запускать с разными конфигурациями;
1. Во время работы коннектора можно выполнять SMT - Single Message Transformer для редактирования данных.

### Задание

1. Создать таблицу `movies`
```sql
CREATE TABLE IF NOT EXISTS movies(
    id INT PRIMARY KEY,
    name VARCHAR(255),
    rank INT,
    released_year INT,
    rating NUMERIC(3, 1),
    personal_raiting NUMERIC(3, 1)
);
```
1. Наполнить таблицу данными из [Кинопоиск TOP 250](https://www.kinopoisk.ru/lists/movies/top250). Количество фильмов выбрать самостоятельно.

```sql
INSERT INTO movies VALUES (1, 'Зелёная миля', 1, 1999, 9.1, 7.0);
```
1. Создать source коннектор, который:
    - [скроет](https://docs.confluent.io/platform/current/connect/transforms/maskfield.html) поле `personal_raiting`;
    - сделает id ключом сообщения.
1. Создать sink коннектор, который:
    - создаст таблицу `sink_movies`;
    - укажет, что `id` - это первичный ключ таблицы `sink_movies`.

## Change Data Capture - Debezium

### Мотивация

Данные в системах постоянно меняются, а анализ данных выполняется после того, как ETL процесс перенесет текущее состояние данных в аналитическую систему, где можно будет их обработать в удобной форме. Аналитикам приходится ждать, когда новые данные появятся в системе, а время, затрачиваемое на ETL, пропорционально объему данных в исходной базе данных и неизбежно только возрастает. Также аналитики часто работают с устаревшим состоянием, но это допущение не всегда применимо на практике.

Для ускорения обработки новых данных применяется подход под названием *Change Data Capture (CDC)*. CDC - это процесс непрерывного отслеживания и захвата изменений в данных. Так на любое изменение данных генерируется событие, в котором, обычно, содержится информация о том, как данные выглядят после изменений. Аналитики могут получать информацию об изменениях в реальном времени, что повышает скорость реакции бизнеса.

Лидером в области CDC является продукт с открытм кодом [Debezium](https://debezium.io/), который развивается при поддержке Red Hat. Архитектурно Debezium представляет коннектор к Apache Kafka.

Важно отметить, что Debezium работает исключительно с базами данных. Debezium имеет поддержку большого числа [реляционных баз данных](https://debezium.io/documentation/faq/#what_databases_can_debezium_monitor), но некоторые из них могут требовать дополнительной конфигурации.

### Конфигурация PostgreSQL

Для корректной работы debesium необходимо установить опцию `wat_level` ([write ahead log](https://www.postgresql.org/docs/current/runtime-config-wal.html)) в зачение `logical`:

In [ ]:
psql -U postgres -d postgres <<EOF
SHOW wal_level;
ALTER SYSTEM SET wal_level = logical
EOF

Внесение изменений на уровне системы требует перезапуска сервера базы данных:

In [ ]:
docker compose restart postgres

Необходимо дождаться, когда postgres станет доступным снова:

In [ ]:
docker compose ps postgres | grep -q healthy || {
    echo "Postgres запускается, пожалуйста, подождите" >&2
    false
} && {
    echo "Postgres готов к работе"
}

Убедиться, что изменения применились:

In [ ]:
psql -U postgres -d postgres <<EOF
SHOW wal_level
EOF

### Получение списка коннекторов Debezium

Debezium предлагает REST API, а поэтому можно получить список зарегистрированных коннекторов при помощи REST запроса:

In [ ]:
curl http://debezium:8083/connectors

### Регистрация коннектора Debezium

In [ ]:
cat <<EOF > /tmp/debezium-postgres-from-products.cfg
{
  "name": "postgres-products-connector",
  "config": {
    "connector.class": "io.debezium.connector.postgresql.PostgresConnector",
    "plugin.name": "pgoutput",
    "database.hostname": "postgres",
    "database.port": "5432",
    "database.user": "postgres",
    "database.password": "postgres",
    "database.dbname": "postgres",
    "database.server.name": "postgres",
    "table.include.list": "public.products",
    "topic.prefix": "debezium-cdc-"
  }
}
EOF

In [ ]:
curl -s http://debezium:8083/connectors \
    -X POST \
    -H 'Content-type: application/json' \
    -d '@/tmp/debezium-postgres-from-products.cfg' | json_pp

In [ ]:
curl -s http://debezium:8083/connectors

### Демонстрация работы Debezium

Для демонстрации необходимо внести изменения в данные:

In [ ]:
psql -U postgres -d postgres <<EOF
insert into products values(5, 'Bacon', 11);
insert into products values(6, 'Ham', 10);
UPDATE products SET price = price * 0.9 WHERE name = 'Eggs';
DELETE FROM products WHERE name = 'Ham';
EOF

После внесения изменений в таблицу можно будет увидеть, что в топике `debezium-cdc-.public.products` появились новые сообщения. Каждое сообщение будет содержать ключ строки, для которой были  изменены данные, а также состояние после внесения изменений. Каждое сообщение содержит исчерпывающую информацию, и, в какой-то мере, излишне подробную, поэтому каждое сообщение становится очень большим. В целях обеспечения удобства работы в данном ноутбуке, выводится только одна запись из топика, остальные записи лучше всего посмотреть в [redpanda](http://localhost:8080/topics/debezium-cdc-.public.products), где имеется удобный Web UI для навигации по каждому сообщению.

In [ ]:
kafka-console-consumer \
    --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic 'debezium-cdc-.public.products' \
    --from-beginning \
    --timeout-ms 10000 \
    --max-messages 1 | json_pp

### Вывод

1. Подход Change Data Capture позволяет реагировать на изменения в реальном времени, что повышает устойчивость бизнеса к резким изменениям;
1. Debezium архитектурно представляет собой коннектор к Apache Kafka;
1. Работа с Debezium может потребовать конфигурации базы данных, что может вызывать затруднения для уже запущенных баз данных.

### Задание

1. Убедиться, что оба коннектора (таблицы `movies`, `sink_movies`) из предыдущего раздела активны;
1. Настроить debezium коннектор для таблицы `sink_movies`;
1. Вставить и обновить и удалить данные в `movies`;
1. Убедиться, что через два коннектора (source, sink) для таблиц `movies` и `sink_movies` и debezium коннектора изменения логируются в kafka топик.